### **LEITURA E FORMATAÇÃO DO ARQUIVO**

In [1]:
import pandas as pd
from pyspark.sql.functions import *
import numpy as np
from pyspark.sql.functions import *
from pyspark.sql import SparkSession
spark = SparkSession.builder.appName("CSV_Reader").getOrCreate()
import matplotlib.pyplot as plt
import seaborn as sns
from pyspark.sql.functions import udf 
from pyspark.sql.types import StringType 


In [2]:
# Conexão JDBC ---


POSTGRES_JDBC_DRIVER_PATH = "C:\\spark\\spark-3.4.1\\jars\\postgresql-42.7.8.jar"


spark = SparkSession.builder \
    .appName("PostgreSQLConnection_Tech3") \
    .config("spark.jars", POSTGRES_JDBC_DRIVER_PATH) \
    .getOrCreate()

print("SparkSession iniciada e driver JDBC 'postgresql-42.7.8.jar' carregado com sucesso.")


print(f"Drivers carregados: {spark.sparkContext.getConf().get('spark.jars')}")

jdbc_url = "jdbc:postgresql://localhost:5432/Tech3" 
jdbc_user = "postgres" 
jdbc_password = "alef"
MODE = "overwrite" 

connection_properties = {
    "user": jdbc_user,
    "password": jdbc_password,

    "driver": "org.postgresql.Driver" 
}

print("Propriedades de conexão definidas.")

SparkSession iniciada e driver JDBC 'postgresql-42.7.8.jar' carregado com sucesso.
Drivers carregados: None
Propriedades de conexão definidas.


In [3]:
csv_path_julho = "PNAD_COVID_072020\PNAD_COVID_072020.csv"

df_julho = spark.read.csv(csv_path_julho, sep = ",", header=True, inferSchema=True)

In [4]:
csv_path_setembro = "PNAD_COVID_092020\PNAD_COVID_092020.csv"

df_setembro = spark.read.csv(csv_path_setembro, sep = ",", header=True, inferSchema=True)

In [5]:
csv_path_novembro = "PNAD_COVID_112020\PNAD_COVID_112020.csv"

df_novembro = spark.read.csv(csv_path_novembro, sep = ",", header=True, inferSchema=True)

In [6]:
# Seleciona apenas as colunas desejadas

colunas_desejadas = [
"A001","UPA","V1008","V1013","A002","A003","A006","B0015","B0011","B00111","B0012","B0014","B0018","B0019","B009F","B009D","B009B","B011","C001","C007C","C007E","C012","V1022","UF"]

df_novembro = df_novembro.select(colunas_desejadas)
df_setembro = df_setembro.select(colunas_desejadas)
df_julho = df_julho.select(colunas_desejadas)

In [7]:
#Junta os três meses em um unico DataFrame
df_completo = df_novembro.union(df_setembro).union(df_julho)



In [8]:

# Renomeia as colunas (exemplo: ajuste conforme necessário)
df_renamed = df_completo.withColumnRenamed("UF", "PESS_UF"
).withColumnRenamed("V1008", "CTRL_NUM_DOMICILIO"
).withColumnRenamed("V1013", "CTRL_MES"
).withColumnRenamed("V1022", "PESS_SIT_DOMICILIO"
).withColumnRenamed("UPA", "CTRL_UPA"
).withColumnRenamed("A001", "CTRL_NUM_MORADOR"
).withColumnRenamed("A002", "PESS_IDADE_MORADOR"
).withColumnRenamed("A003", "PESS_SEXO_MORADOR"
).withColumnRenamed("A006", "PESS_ESTUDA"
).withColumnRenamed("B0015", "SINT_DOR_CABECA"   
).withColumnRenamed("B0011", "SINT_FEBRE"
).withColumnRenamed("B0012", "SINT_TOSSE"
).withColumnRenamed("B0014", "SINT_DIF_RESPIRAR"
).withColumnRenamed("B0018", "SINT_CORIZA"
).withColumnRenamed("B0019", "SINT_FADIGA"
).withColumnRenamed("B00111", "SINT_PERDA_OLFATO"
).withColumnRenamed("B009F", "TESTE_SANGUE_VEIA"
).withColumnRenamed("B009D", "TESTE_SANGUE_DEDO"
).withColumnRenamed("B009B", "TESTE_SWAB"			   
).withColumnRenamed("B011", "DISTANC_SOCIAL"
).withColumnRenamed("C001", "TRAB_SP_TRABALHOU"
).withColumnRenamed("C007C", "TRAB_TIPO_TRABALHO"
).withColumnRenamed("C007E", "TRAB_QUANT_EMPREGADOS"
).withColumnRenamed("C012", "TRAB_PRESENCIAL"
                    )

df_renamed = df_renamed.withColumn(
    "CTRL_ID",
    concat(
        lpad(df_renamed["CTRL_UPA"], 15, "0"),
        lpad(df_renamed["CTRL_NUM_DOMICILIO"], 2, "0"),
        lpad(df_renamed["CTRL_NUM_MORADOR"], 2, "0")
    )
)


# Exibe o DataFrame renomeado
df_renamed.show()

+----------------+---------+------------------+--------+------------------+-----------------+-----------+---------------+----------+-----------------+----------+-----------------+-----------+-----------+-----------------+-----------------+----------+--------------+-----------------+------------------+---------------------+---------------+------------------+-------+-------------------+
|CTRL_NUM_MORADOR| CTRL_UPA|CTRL_NUM_DOMICILIO|CTRL_MES|PESS_IDADE_MORADOR|PESS_SEXO_MORADOR|PESS_ESTUDA|SINT_DOR_CABECA|SINT_FEBRE|SINT_PERDA_OLFATO|SINT_TOSSE|SINT_DIF_RESPIRAR|SINT_CORIZA|SINT_FADIGA|TESTE_SANGUE_VEIA|TESTE_SANGUE_DEDO|TESTE_SWAB|DISTANC_SOCIAL|TRAB_SP_TRABALHOU|TRAB_TIPO_TRABALHO|TRAB_QUANT_EMPREGADOS|TRAB_PRESENCIAL|PESS_SIT_DOMICILIO|PESS_UF|            CTRL_ID|
+----------------+---------+------------------+--------+------------------+-----------------+-----------+---------------+----------+-----------------+----------+-----------------+-----------+-----------+-----------------+---

### **DICIONÁRIOS DE DOMINIO DOS DADOS**

In [9]:
csv_path_dict = "Dicionario_PNAD_COVID_112020_20220621.csv"

# pula as 4 primeiras linhas e usa a 5ª como cabeçalho
df_dict = pd.read_csv(
    csv_path_dict, 
    sep=";", 
    encoding="latin1", 
    skiprows=4,   # pula as 4 primeiras
    header=0      # a próxima linha (linha 5) vira o cabeçalho
)

# Indices das linhas de comentários do dicionário 
indices_to_remove = [91, 169, 343, 569, 598, 614]

df_dict_filtered = df_dict.drop(indices_to_remove)

# Usar ffill para preencher os valores vazios com o valor anterior
df_dict_filtered['Ano'] = df_dict_filtered['Ano'].ffill()
df_dict_filtered['Ano de referência'] = df_dict_filtered['Ano de referência'].ffill()

# Criando DF spark
df_dict_filtered = spark.createDataFrame(df_dict_filtered)


#Ronomeando colunaaa
df_dict_rename = df_dict_filtered.withColumnRenamed("Ano", "COD_VAR"
).withColumnRenamed("Ano de referência", "DESCR_VAR"
).withColumnRenamed("Unnamed: 4", "COD_TIPO"
).withColumnRenamed("Unnamed: 5", "DESCR_TIPO")

df_dict_rename = df_dict_rename.select(
    "COD_VAR", "DESCR_VAR","COD_TIPO","DESCR_TIPO"
)

#FILTRANDO APENAS COLUNAS DESEJADAS
df_filtro = df_dict_rename.filter(col("COD_VAR").isin(colunas_desejadas))

df_filtro.show()


+-------+--------------------+--------+-------------------+
|COD_VAR|           DESCR_VAR|COD_TIPO|         DESCR_TIPO|
+-------+--------------------+--------+-------------------+
|     UF|Unidade da Federação|      11|           Rondônia|
|     UF|Unidade da Federação|      12|               Acre|
|     UF|Unidade da Federação|      13|           Amazonas|
|     UF|Unidade da Federação|      14|            Roraima|
|     UF|Unidade da Federação|      15|               Pará|
|     UF|Unidade da Federação|      16|              Amapá|
|     UF|Unidade da Federação|      17|          Tocantins|
|     UF|Unidade da Federação|      21|           Maranhão|
|     UF|Unidade da Federação|      22|              Piauí|
|     UF|Unidade da Federação|      23|              Ceará|
|     UF|Unidade da Federação|      24|Rio Grande do Norte|
|     UF|Unidade da Federação|      25|            Paraíba|
|     UF|Unidade da Federação|      26|         Pernambuco|
|     UF|Unidade da Federação|      27| 

In [10]:
dic_pess_uf_df = (
    df_filtro
    .filter(col("COD_VAR") == "UF")
    .select("COD_TIPO", "DESCR_TIPO")
    .withColumnRenamed("COD_TIPO", "PESS_UF")
    .withColumnRenamed("DESCR_TIPO", "PESS_UF_DESC"))



dic_pess_sit_domicilio_df = (
    df_filtro
    .filter(col("COD_VAR") == "V1022")
    .select("COD_TIPO", "DESCR_TIPO")
    .withColumnRenamed("COD_TIPO", "PESS_SIT_DOMICILIO")
    .withColumnRenamed("DESCR_TIPO", "PESS_SIT_DOMICILIO_DESC"))


dic_pess_sexo_morador = (
    df_filtro
    .filter(col("COD_VAR") == "A003")
    .select("COD_TIPO", "DESCR_TIPO")
    .withColumnRenamed("COD_TIPO", "PESS_SEXO_MORADOR")
    .withColumnRenamed("DESCR_TIPO", "PESS_SEXO_MORADOR_DESC"))



dic_sintomas_df = (
    df_filtro
    .filter(col("COD_VAR")=='B0012')
    .select("COD_TIPO", "DESCR_TIPO")
    .withColumnRenamed("COD_TIPO", "SINT_COD")
    .withColumnRenamed("DESCR_TIPO", "SINT_DESC"))

dic_estuda_df = (
    df_filtro
    .filter(col("COD_VAR")=='A006')
    .select("COD_TIPO", "DESCR_TIPO")
    .withColumnRenamed("COD_TIPO", "ESTUDA_COD")
    .withColumnRenamed("DESCR_TIPO", "ESTUDA_DESC"))    

dic_resultado_teste_df = (
    df_filtro
    .filter(col("COD_VAR")=='B009B')
    .select("COD_TIPO", "DESCR_TIPO")
    .withColumnRenamed("COD_TIPO", "TESTE_COD")
    .withColumnRenamed("DESCR_TIPO", "TESTE_DESC"))    

dic_isolamento_social_df = (
    df_filtro
    .filter(col("COD_VAR")=='B011')
    .select("COD_TIPO", "DESCR_TIPO")
    .withColumnRenamed("COD_TIPO", "ISO_COD")
    .withColumnRenamed("DESCR_TIPO", "ISO_DESC"))    

dic_trab_sp_trabalhou_df = (
    df_filtro
    .filter(col("COD_VAR")=='C001')
    .select("COD_TIPO", "DESCR_TIPO")
    .withColumnRenamed("COD_TIPO", "TRAB_SP_TRABALHOU_COD")
    .withColumnRenamed("DESCR_TIPO", "TRAB_SP_TRABALHOU_DESC"))

dic_trab_tipo_trabalho_df = (
    df_filtro
    .filter(col("COD_VAR")=='C007C')
    .select("COD_TIPO", "DESCR_TIPO")
    .withColumnRenamed("COD_TIPO", "TRAB_TIPO_TRABALHO_COD")
    .withColumnRenamed("DESCR_TIPO", "TRAB_TIPO_TRABALHO_DESC"))

dic_trab_qt_empregados_df = (
    df_filtro
    .filter(col("COD_VAR")=='C007E')
    .select("COD_TIPO", "DESCR_TIPO")
    .withColumnRenamed("COD_TIPO", "TRAB_QUANT_EMPREGADOS_COD")
    .withColumnRenamed("DESCR_TIPO", "TRAB_QUANT_EMPREGADOS_DESC"))


dic_trab_ms_local_df = (
    df_filtro
    .filter(col("COD_VAR")=='C012')
    .select("COD_TIPO", "DESCR_TIPO")
    .withColumnRenamed("COD_TIPO", "TRAB_MESMO_LOCAL_COD")
    .withColumnRenamed("DESCR_TIPO", "TRAB_MESMO_LOCAL_DESC"))



### **SEPARAÇÃO DOS DADOS NAS TABELAS DISTINTAS**

In [11]:
colunas_dados_pess = ["CTRL_MES", "CTRL_ID", "PESS_UF", "PESS_SIT_DOMICILIO", "PESS_IDADE_MORADOR", "PESS_SEXO_MORADOR","PESS_ESTUDA"]

df_dados_pess = df_renamed.select(colunas_dados_pess)

colunas_dados_trab = ["CTRL_MES", "CTRL_ID", "TRAB_SP_TRABALHOU", "TRAB_TIPO_TRABALHO", "TRAB_QUANT_EMPREGADOS", "TRAB_PRESENCIAL"]

df_dados_trab = df_renamed.select(colunas_dados_trab)

colunas_dados_sint = ["CTRL_MES", "CTRL_ID",  "SINT_FEBRE", "SINT_TOSSE", "SINT_DOR_CABECA", "SINT_DIF_RESPIRAR", "SINT_CORIZA", "SINT_FADIGA", "SINT_PERDA_OLFATO"]

df_dados_sint = df_renamed.select(colunas_dados_sint)

colunas_dados_iso_social = ["CTRL_MES", "CTRL_ID", "DISTANC_SOCIAL"]

df_dados_iso_social = df_renamed.select(colunas_dados_iso_social)


colunas_dados_tst_covd = ["CTRL_MES", "CTRL_ID","TESTE_SWAB", "TESTE_SANGUE_DEDO", "TESTE_SANGUE_VEIA"]

df_dados_dados_tst_covd = df_renamed.select(colunas_dados_tst_covd)


In [12]:
df_dados_pess.printSchema()
df_dados_trab.printSchema()
df_dados_sint.printSchema()
df_dados_iso_social.printSchema()
df_dados_dados_tst_covd.printSchema()


root
 |-- CTRL_MES: integer (nullable = true)
 |-- CTRL_ID: string (nullable = true)
 |-- PESS_UF: integer (nullable = true)
 |-- PESS_SIT_DOMICILIO: integer (nullable = true)
 |-- PESS_IDADE_MORADOR: integer (nullable = true)
 |-- PESS_SEXO_MORADOR: integer (nullable = true)
 |-- PESS_ESTUDA: integer (nullable = true)

root
 |-- CTRL_MES: integer (nullable = true)
 |-- CTRL_ID: string (nullable = true)
 |-- TRAB_SP_TRABALHOU: integer (nullable = true)
 |-- TRAB_TIPO_TRABALHO: integer (nullable = true)
 |-- TRAB_QUANT_EMPREGADOS: integer (nullable = true)
 |-- TRAB_PRESENCIAL: integer (nullable = true)

root
 |-- CTRL_MES: integer (nullable = true)
 |-- CTRL_ID: string (nullable = true)
 |-- SINT_FEBRE: integer (nullable = true)
 |-- SINT_TOSSE: integer (nullable = true)
 |-- SINT_DOR_CABECA: integer (nullable = true)
 |-- SINT_DIF_RESPIRAR: integer (nullable = true)
 |-- SINT_CORIZA: integer (nullable = true)
 |-- SINT_FADIGA: integer (nullable = true)
 |-- SINT_PERDA_OLFATO: integer 

In [13]:
# Tabela de Dados Pessoais
df_dados_pess.createOrReplaceTempView("dados_pessoais")

# Tabela de Dados de Trabalho
df_dados_trab.createOrReplaceTempView("dados_trabalho")

# Tabela de Dados de Sintomas
df_dados_sint.createOrReplaceTempView("dados_sintomas")

# Tabela de Dados de Isolamento Social
df_dados_iso_social.createOrReplaceTempView("dados_isolamento_social")

# Tabela de Dados de Teste COVID
df_dados_dados_tst_covd.createOrReplaceTempView("dados_teste_covid")

# Tabela Dicionário de Unidades Federativas (UF)
dic_pess_uf_df.createOrReplaceTempView("dic_uf")

# Tabela Dicionário de Situação do Domicílio
dic_pess_sit_domicilio_df.createOrReplaceTempView("dic_sit_domicilio")

# Tabela Dicionário de Sexo do Morador
dic_pess_sexo_morador.createOrReplaceTempView("dic_sexo_morador")

# Tabela Dicionário de Sintomas
dic_sintomas_df.createOrReplaceTempView("dic_sintomas")

# Tabela Dicionário de Estuda
dic_estuda_df.createOrReplaceTempView("dic_estuda")

# Tabela Dicionário de Resultado de Teste
dic_resultado_teste_df.createOrReplaceTempView("dic_resultado_teste")

# Tabela Dicionário de Isolamento Social
dic_isolamento_social_df.createOrReplaceTempView("dic_isolamento_social")

# Tabela Dicionário de Trabalhou na Semana Passada (SP)
dic_trab_sp_trabalhou_df.createOrReplaceTempView("dic_trab_sp_trabalhou")

# Tabela Dicionário de Tipo de Trabalho
dic_trab_tipo_trabalho_df.createOrReplaceTempView("dic_trab_tipo_trabalho")

# Tabela Dicionário de Quantidade de Empregados
dic_trab_qt_empregados_df.createOrReplaceTempView("dic_trab_qt_empregados")

# Tabela Dicionário de Mesma Localidade (Trabalho)
dic_trab_ms_local_df.createOrReplaceTempView("dic_trab_mesmo_local")


### **CRIAÇÃO DO BANCO DE DADOS**

In [14]:
def inserir_tabela_postgres(df, nome_tabela):
    """Insere um DataFrame (df) em uma tabela (nome_tabela) no PostgreSQL."""
    print(f"   -> Tentando inserir tabela: '{nome_tabela}'...")
    try:
        df.write.jdbc(
            url=jdbc_url,
            table=nome_tabela,
            mode=MODE,
            properties=connection_properties
        )
        print(f"✅ Tabela '{nome_tabela}' inserida com sucesso (Modo: {MODE}).")
    except Exception as e:
        # A exceção Py4JJavaError será capturada aqui
        print(f"❌ ERRO GRAVE ao inserir a tabela '{nome_tabela}'. Verifique o driver JDBC e a conexão do DB.")
        print(f"Detalhes do Erro: {e}")





print("\n--- INICIANDO INSERÇÃO DOS DADOS PRINCIPAIS ---")
for nome_tabela, df in dfs_dados.items():
    inserir_tabela_postgres(df, nome_tabela)

print("\n--- INICIANDO INSERÇÃO DOS DICIONÁRIOS (DICs) ---")
for nome_tabela, df in dfs_dics.items():
    inserir_tabela_postgres(df, nome_tabela)


--- INICIANDO INSERÇÃO DOS DADOS PRINCIPAIS ---


NameError: name 'dfs_dados' is not defined

### **ANALISES EXPLORATORIAS**


### 1 - Estados com mais testes positivos

### 2 - Positivos por área (Urbano/Rural)

### 3 - Faixas etárias

### 4 - Homens vs Mulheres Infectados

### 5 - Estudantes ?

### 6 - Testes Positivos por tipo de Trabalho 

### 7 -sintomas mais apresentados ?

### 8 - Assintomáticos positivos?

### 9 Pessoas que praticaram Distanciamento Social para pessoas com SINTOMÁTICOS

### 9 Pessoas que praticaram Distanciamento Social para pessoas com ASSINTOMÁTICOS

### 10 Casos Positivos por Situação que  Trabalha

### 11 - Qual teste mais feito por todos

### 12 - Casos Positivos: Faixa Etária vs. Tipo de Ocupação

### 13 - Proporção: Sintomáticos vs. Assintomáticos por idade

### 14 - Casos Positivos por local de trabalho
